## Optimization exercise

**Goal: Train the 2nd order polynomial predictor using both gradient descent and stochastic gradient descent. Optimize the stepsizes and compare against scikit-learn's implementation.**

1. Download the data from [here](https://drive.google.com/file/d/0Bz9_0VdXvv9bUUNlUTVrMF9VcVU/view?usp=sharing).


2. Create a function psi(x), which transforms features AST (assists), REB (rebounds) and STL (steals) into 2nd order polynomial features (add each feature squared and each pair of features multiplied with every other)


3. Create a transformed data matrix X, where each x is mapped to psi(x).


4. Create a function p2(x,w), which outputs the value of the polynomial at x for given parameters w.


5. Create a function Loss(X,y,w), which computes the squared loss of predicting y from X by p2(x,w) using parameters w. Take variable PTS as y. We will predict scored points based on assists, rebounds and steals.


6. Code up the gradient descent. It should input a point w and a stepsize.


7. Choose an arbitrary point and stepsize. Run gradient descent for 100 iterations and compute the Loss after each iteration. How does the loss behave? Does it converge to something?


8. Can you find the stepsize, for which the loss is smallest after 100 iterations?

In [1]:
# IMPORT PACKAGES
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [2]:
nb = pd.read_csv('nba_games_2013_2015.csv', delimiter=';')
x = nb[['AST','REB','STL']]
y = nb['PTS']

In [3]:
nb

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22015,1610612750,MIN,Minnesota Timberwolves,21501226,2016-04-13,MIN vs. NOP,W,240,144,...,0.826,5,38,43,41,14,8,13,20,35.0
1,22015,1610612749,MIL,Milwaukee Bucks,21501225,2016-04-13,MIL vs. IND,L,240,92,...,0.846,7,36,43,23,8,3,15,15,-5.0
2,22015,1610612738,BOS,Boston Celtics,21501217,2016-04-13,BOS vs. MIA,W,240,98,...,0.864,10,29,39,20,7,3,7,20,10.0
3,22015,1610612747,LAL,Los Angeles Lakers,21501228,2016-04-13,LAL vs. UTA,W,239,101,...,0.867,8,39,47,19,6,3,13,17,5.0
4,22015,1610612739,CLE,Cleveland Cavaliers,21501220,2016-04-13,CLE vs. DET,L,265,110,...,0.733,8,35,43,21,4,7,10,23,-2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7375,22013,1610612753,ORL,Orlando Magic,21300001,2013-10-29,ORL @ IND,L,241,87,...,0.600,13,26,39,17,10,6,17,26,-10.0
7376,22013,1610612748,MIA,Miami Heat,21300002,2013-10-29,MIA vs. CHI,W,239,107,...,0.759,5,35,40,26,10,7,18,21,12.0
7377,22013,1610612747,LAL,Los Angeles Lakers,21300003,2013-10-29,LAL vs. LAC,W,240,116,...,0.643,18,34,52,23,8,6,19,23,13.0
7378,22013,1610612741,CHI,Chicago Bulls,21300002,2013-10-29,CHI @ MIA,L,238,95,...,0.783,11,30,41,23,11,4,18,27,-12.0


In [4]:
x

,AST,REB,STL
0,41,43,14
1,23,43,8
2,20,39,7
3,19,47,6
4,21,43,4
...,...,...,...
7375,17,39,10
7376,26,40,10
7377,23,52,8
7378,23,41,11


In [5]:
x2 = x.copy()
x2

,AST,REB,STL
0,41,43,14
1,23,43,8
2,20,39,7
3,19,47,6
4,21,43,4
...,...,...,...
7375,17,39,10
7376,26,40,10
7377,23,52,8
7378,23,41,11


In [66]:
def psi(x):
    x_1 = x.copy()
    
    x_1['constant'] = 1
    x_1['AST*AST'] = x_1['AST']*x_1['AST']
    x_1['REB*REB'] = x_1['REB']*x_1['REB']
    x_1['STL*STL'] = x_1['STL']*x_1['STL']
    x_1['AST*REB'] = x_1['AST']*x_1['REB']
    x_1['AST*STL'] = x_1['AST']*x_1['STL']
    x_1['REB*STL'] = x_1['REB']*x_1['STL']
    
    #x_1 = x_1.drop(columns=['AST','REB','STL'])
    
    return x_1

3. Create a transformed data matrix X, where each x is mapped to psi(x).

In [67]:
# x^0 + x^1 + x^2

x3 = psi(x2)
x3

,AST,REB,STL,constant,AST*AST,REB*REB,STL*STL,AST*REB,AST*STL,REB*STL
0,41,43,14,1,1681,1849,196,1763,574,602
1,23,43,8,1,529,1849,64,989,184,344
2,20,39,7,1,400,1521,49,780,140,273
3,19,47,6,1,361,2209,36,893,114,282
4,21,43,4,1,441,1849,16,903,84,172
...,...,...,...,...,...,...,...,...,...,...
7375,17,39,10,1,289,1521,100,663,170,390
7376,26,40,10,1,676,1600,100,1040,260,400
7377,23,52,8,1,529,2704,64,1196,184,416
7378,23,41,11,1,529,1681,121,943,253,451


4. Create a function p2(x,w), which outputs the value of the polynomial at x for given parameters w.

In [68]:
def p2(x,w):    
    
    w = np.array(w)
    
    x_psi = x * w
    
    x_psi['p2(x,w)'] = x_psi.sum(axis=1)
    
    return x_psi

In [69]:
p2(x3,[1,1,1,1,2,3,2,1,2,3])

,AST,REB,STL,constant,AST*AST,REB*REB,STL*STL,AST*REB,AST*STL,REB*STL,"p2(x,w)"
0,41,43,14,1,3362,5547,392,1763,1148,1806,14117
1,23,43,8,1,1058,5547,128,989,368,1032,9197
2,20,39,7,1,800,4563,98,780,280,819,7407
3,19,47,6,1,722,6627,72,893,228,846,9461
4,21,43,4,1,882,5547,32,903,168,516,8117
...,...,...,...,...,...,...,...,...,...,...,...
7375,17,39,10,1,578,4563,200,663,340,1170,7581
7376,26,40,10,1,1352,4800,200,1040,520,1200,9189
7377,23,52,8,1,1058,8112,128,1196,368,1248,12194
7378,23,41,11,1,1058,5043,242,943,506,1353,9221


5. Create a function Loss(X,y,w), which computes the squared loss of predicting y from X by p2(x,w) using parameters w. Take variable PTS as y. We will predict scored points based on assists, rebounds and steals.

In [12]:
def loss(X,y,w):    
    loss_df = p2(X,w)
    
    loss_df['sq_loss'] = (y - loss_df['p2(x,w)'])**2
    
    return loss_df

In [70]:
loss(x3,nb['PTS'],[1,1,1,1,2,3,2,1,2,3])

,AST,REB,STL,constant,AST*AST,REB*REB,STL*STL,AST*REB,AST*STL,REB*STL,"p2(x,w)",sq_loss
0,41,43,14,1,3362,5547,392,1763,1148,1806,14117,195244729
1,23,43,8,1,1058,5547,128,989,368,1032,9197,82901025
2,20,39,7,1,800,4563,98,780,280,819,7407,53421481
3,19,47,6,1,722,6627,72,893,228,846,9461,87609600
4,21,43,4,1,882,5547,32,903,168,516,8117,64112049
...,...,...,...,...,...,...,...,...,...,...,...,...
7375,17,39,10,1,578,4563,200,663,340,1170,7581,56160036
7376,26,40,10,1,1352,4800,200,1040,520,1200,9189,82482724
7377,23,52,8,1,1058,8112,128,1196,368,1248,12194,145878084
7378,23,41,11,1,1058,5043,242,943,506,1353,9221,83283876


6. Code up the gradient descent. It should input a point w and a stepsize.

In [ ]:
# X = x3
# y = nb['PTS']
# alpha = 0.01
# w = [1,2,3,2,1,2,3]
# max_iters = 10

In [59]:
def grad_descent(X, y, alpha, w, max_iters):
    # Expected input: X should be all x values from the polynomial equation
    
    w = np.array(w)
    
    df = X.copy()
    
    #number of observation in X and y
    m = len(y)
    loss_func_history = []
    
    # computing initial loss
    loss_func_history.append(loss(X,y,w)['sq_loss'])
    
    for k in range(max_iters):
        
        # compute predictions with actual w        
        predictions = p2(X, w)['p2(x,w)']
        
        # number of coeffcients
        w_size = len(w)
        # computing gradient of each w separately (approximates partial derivation of function)

        for i in range(w_size):
            #taking 1 variable only from the dataframe
            temp = df.iloc[:, i]
            #errors multiply by the variable in place
            errors_x1 = (predictions - y) * temp
            # adjusting w[i]
            # gradient is basically sum of the errors divided by number of observation, np.mean could be used as well

            w[i] = w[i] - alpha * (1.0 / m) * errors_x1.sum()
       
        loss_func_history.append(loss(X,y,w)['sq_loss'])

    return loss_func_history

In [73]:
loss_func_history = grad_descent(x3,nb['PTS'],alpha=0.01, w=[0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1], max_iters=100)
loss_func_history[-1]

F:\Anaconda\envs\lhl\lib\site-packages\ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars


0       20736.0
1        8464.0
2        9604.0
3       10201.0
4       12100.0
         ...   
7375     7569.0
7376    11449.0
7377    13456.0
7378     9025.0
7379     9409.0
Name: sq_loss, Length: 7380, dtype: float64

In [16]:
def gradient_descent(stepsize,w):
    past_scores = []
    past_Ws = [w]
    
    for i in range(0,100):
        prediction = p2(x3,w)['p2(x,w)']
        
        error = prediction - nb['PTS']
        
        score = 1/(2*7380) * np.dot(error.T, error)
        past_scores.append(score)
        
        w = w - (stepsize * (1/7380) * np.dot(x3.T, error))
        past_Ws.append(w)
    
    return past_Ws, past_scores

7. Choose an arbitrary point and stepsize. Run gradient descent for 100 iterations and compute the Loss after each iteration. How does the loss behave? Does it converge to something?

In [20]:
gradient_descent(0.01, [1,2,3,2,1,2,3])
#past_Ws, past_scores = gradient_descent(0.01, [1,2,3,2,1,2,3])

F:\Anaconda\envs\lhl\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in subtract
  del sys.path[0]


([[1, 2, 3, 2, 1, 2, 3],
  array([-9.01166409e+01, -4.88405692e+04, -1.85784995e+05, -6.51967454e+03,
         -9.20162548e+04, -1.64432924e+04, -3.19558902e+04]),
  array([4.82801680e+06, 2.56324283e+09, 9.94591585e+09, 3.30586533e+08,
         4.87899493e+09, 8.47224937e+08, 1.66389719e+09]),
  array([-2.57524893e+11, -1.36682238e+14, -5.30598174e+14, -1.76275413e+13,
         -2.60227606e+14, -4.51759949e+13, -8.87441220e+13]),
  array([1.37375264e+16, 7.29119129e+18, 2.83045621e+19, 9.40327723e+17,
         1.38816773e+19, 2.40987540e+18, 4.73400478e+18]),
  array([-7.32822119e+20, -3.88945238e+23, -1.50989413e+24, -5.01613523e+22,
         -7.40511754e+23, -1.28553696e+23, -2.52533337e+23]),
  array([3.90920650e+25, 2.07481080e+28, 8.05446208e+28, 2.67583468e+27,
         3.95022651e+28, 6.85763886e+27, 1.34712768e+28]),
  array([-2.08534856e+30, -1.10679845e+33, -4.29661643e+33, -1.42741193e+32,
         -2.10723049e+33, -3.65817648e+32, -7.18619180e+32]),
  array([1.11241977e+35

In [19]:
w = past_Ws[-1]

#Print the results...
print("Gradient Descent: {:.2f}, {:.2f}".format(w[0], w[1]))

Gradient Descent: nan, nan


F:\Anaconda\envs\lhl\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in subtract
  del sys.path[0]


8. Can you find the stepsize, for which the loss is smallest after 100 iterations?

In [74]:
# Do 100 iterations per step size

losses = []
steps = [1, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001, 0.0000001, 0.00000001]
for alpha in steps:
    print(f"Step: {alpha}")
    losses.append(grad_descent(x3,nb['PTS'],alpha=0.01, w=[0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1], max_iters=100))

Step: 1


F:\Anaconda\envs\lhl\lib\site-packages\ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars


Step: 0.1
Step: 0.01
Step: 0.001
Step: 0.0001
Step: 1e-05
Step: 1e-06
Step: 1e-07
Step: 1e-08


In [75]:
# All seem to converge to the same values

for list in losses:
    print(list[-1])

0       20736.0
1        8464.0
2        9604.0
3       10201.0
4       12100.0
         ...   
7375     7569.0
7376    11449.0
7377    13456.0
7378     9025.0
7379     9409.0
Name: sq_loss, Length: 7380, dtype: float64
0       20736.0
1        8464.0
2        9604.0
3       10201.0
4       12100.0
         ...   
7375     7569.0
7376    11449.0
7377    13456.0
7378     9025.0
7379     9409.0
Name: sq_loss, Length: 7380, dtype: float64
0       20736.0
1        8464.0
2        9604.0
3       10201.0
4       12100.0
         ...   
7375     7569.0
7376    11449.0
7377    13456.0
7378     9025.0
7379     9409.0
Name: sq_loss, Length: 7380, dtype: float64
0       20736.0
1        8464.0
2        9604.0
3       10201.0
4       12100.0
         ...   
7375     7569.0
7376    11449.0
7377    13456.0
7378     9025.0
7379     9409.0
Name: sq_loss, Length: 7380, dtype: float64
0       20736.0
1        8464.0
2        9604.0
3       10201.0
4       12100.0
         ...   
7375     7569.0
7376    